In [12]:
for key in USABLE_FRAMES.keys():
    for base in MATCHING_RESULTS.keys():
        SS = {
            "X": 0,
            "Z": 0
        }
        for i in range(len(key)):
            if base[i] == "X":
                bit = int(key[i].split(",")[0][0])
            else:
                bit = int(key[i].split(",")[1][0])
                
            SS[base[i]] ^= bit

        ss = ''.join(map(str, SS.values()))
        
        if base == ("X", "X", "X") or base == ("Z", "Z", "Z") or base == ("X", "Z", "X") or base == ("Z", "X", "Z"):
            ss += "0"
        else:
            ss += "1"

        MATCHING_RESULTS_DERIVATION[USABLE_FRAMES[key]][ss] = MATCHING_RESULTS[base]

In [1]:
from alice import Alice
from bob import Bob

def execute_qkd(pairs, DEBUG = True, depolarize_probability = 0):
    # Initialization
    alice = Alice(pairs)
    bob = Bob(depolarize_probability)
    
    # Send pairs
    pairs = alice.prepare()

    # Measure each pair and retrieve double matchings
    double_matchings = bob.measure(pairs)
    
    # Verify that there's double matchings
    if not len(double_matchings):
        print("Key exchange failed, there's no double matchings. Please, try again.")
        return (), ()
    
    # Compute usable frames for Bob
    usable_frames, usable_frames_types = alice.compute_usable_frames(double_matchings)
    
    # Veirfy that there's usable frames
    if not len(usable_frames):
        print("Key exchange failed, there's no usable frames. Please, try again.")
        return (), ()

    # Compute sifting string
    sifting_string = bob.compute_sifting_string(usable_frames)

    # Compute measured string
    measured_string = bob.compute_measured_string(usable_frames)

    # Bob computes secret key
    bob_key = bob.generate_shared_key(usable_frames)

    # Alice computes secret key
    alice_key = alice.generate_shared_key(usable_frames_types, sifting_string)
    
    if alice_key == bob_key:
        if DEBUG:
            print("Key exchange completed, here's the data:")
            print(f"Alice send pairs: {alice.pairs_data}")
            print(f"Bob send double matchings: {double_matchings}")
            print(f"Alice send usable frames: {usable_frames}")
            print(f"Bob send sifting string: {sifting_string}")
            print(f"Bob shared key: {bob_key}")
            print(f"Alice shared key: {alice_key}")
        return (usable_frames, sifting_string, measured_string), (alice_key, bob_key)
    else:
        print("Key exchange failed, both keys are not equal. Please, try again.")
        return (usable_frames, sifting_string, measured_string), (alice_key, bob_key)

In [2]:
public, private = execute_qkd(16)

Key exchange completed, here's the data:
Alice send pairs: {0: '1x,0z', 1: '0x,0z', 2: '1x,1z', 3: '0x,1z', 4: '1x,1z', 5: '1x,0z', 6: '1x,1z', 7: '1x,0z', 8: '1x,1z', 9: '0x,1z', 10: '0x,0z', 11: '0x,0z', 12: '0x,1z', 13: '0x,0z', 14: '1x,1z', 15: '0x,0z'}
Bob send double matchings: [1, 2, 3, 4, 5, 12]
Alice send usable frames: [(1, 2, 3), (1, 2, 5), (1, 2, 12), (1, 3, 4), (1, 3, 5), (1, 4, 5), (1, 4, 12), (1, 5, 12), (2, 3, 12), (2, 4, 5), (2, 4, 12)]
Bob send sifting string: ['110', '001', '110', '000', '111', '111', '000', '110', '101', '010', '101']
Bob shared key: 011111011001101101001011110010110
Alice shared key: 011111011001101101001011110010110


In [2]:
from partial_key_recovery import attack

public, private = execute_qkd(64, DEBUG = False)
usable_frames, sifting_string, measured_string = public
alice_key, bob_key = private

assert alice_key == bob_key

key_recovered = attack(usable_frames, sifting_string, measured_string, DEBUG = False)

private_key_blocks = [ alice_key[i : i + 3] for i in range(0, len(private[0]), 3)  ]
key_recovered_blocks = [ key_recovered[i : i + 3] for i in range(0, len(key_recovered), 3)  ]

for x, y in zip(private_key_blocks, key_recovered_blocks):
    print(x, y)

110 110
110 110
010 010
110 110
010    
010 010
010    
010 010
010 010
110 110
010    
110 110
010 010
010 010
010 010
110 111
010    
110    
110    
010    
110    
010    
110    
010    
010    
010    
010    
110    
010    
110    
110    
110    
010    
010    
010    
010    
110    
000    
100    
000    
000    
000    
000    
100    
000    
100    
000    
100    
100    
100 100
100 100
000    
100 100
000    
100 100
000    
000    
000    
000    
100 100
000    
100 100
100 100
100 100
000    
000    
000    
000    
100    
110 110
010 010
110 110
010    
010 010
010    
010 010
010 010
110 110
010    
110 110
010 010
010 010
010 010
110 111
110    
010    
110    
010    
110    
010    
010    
010    
010    
110    
010    
110    
110    
110    
010    
010    
010    
010    
110    
010 010
110 110
010    
010 010
010    
010 010
010 010
110 110
010    
110 110
010 010
010 010
010 010
110 110
100 100
000    
000    
000    
000    
000    
100 100
000    


In [5]:
from partial_key_recovery import attack

public, private = execute_qkd(256, DEBUG = False)
usable_frames, sifting_string, measured_string = public
alice_key, bob_key = private
assert alice_key == bob_key

key_recovered = attack(usable_frames, sifting_string, DEBUG = False)

private_key_blocks = [ alice_key[i : i + 2] for i in range(0, len(private[0]), 2)  ]
key_recovered_blocks = [ key_recovered[i : i + 2] for i in range(0, len(key_recovered), 2)  ]

for x, y in zip(private_key_blocks, key_recovered_blocks):
    print(x, y)

11   
11   
11   
01   
01   
11   
11   
01   
01   
11   
01   
11   
11   
01   
11   
11   
11   
11   
01   
01   
01   
01   
11   
01   
11   
01   
11   
01   
01   
01   
11   
11   
01   
01   
11   
11   
01   
11   
11   
01   
11   
01   
01   
01   
01   
01   
11   
11   
01   
11   
11   
01   
11   
01   
11   
11   
01   
00   
00   
10 10
10 10
00 00
00 00
10   
00 00
00 00
00   
10   
00 00
10   
00   
10   
00   
10   
10 10
10   
10   
00   
10 10
10 10
00   
00   
10   
10   
10   
10 10
00 00
10   
10 10
00 00
00   
00   
00   
00   
10   
00 00
10 10
00 00
00 00
00   
10   
10   
00 00
00   
10   
00   
10 10
10 10
10   
10 10
00 00
00   
10   
10   
10 10
10 10
10   
10   
10   
00   
10 10
10   
00   
00   
00   
10   
00   
10   
00   
10   
00   
10   
00   
10   
10   
10   
10   
00   
10   
00   
10   
00   
10   
10   
10   
10   
10   
10   
10   
00   
10   
00   
10   
00   
10   
10   
10   
00   
00   
00   
10   
10   
00   
10   
10   
10   
10  

In [133]:
def key_recovery_metrics(bits):
    public, private = execute_qkd(bits, DEBUG = False)
    usable_frames, sifting_string = public
    alice_key, bob_key = private
    assert alice_key == bob_key

    key_recovered = attack(usable_frames, sifting_string, DEBUG = False)

    private_key_blocks = [ alice_key[i : i + 2] for i in range(0, len(private[0]), 2)  ]
    key_recovered_blocks = [ key_recovered[i : i + 2] for i in range(0, len(key_recovered), 2)  ]

    bits_recovered = 0
    for x, y in zip(private_key_blocks, key_recovered_blocks):
        if y != "  ":
            bits_recovered += 2
    
    return bits_recovered / len(alice_key)

In [142]:
from numpy import mean

# mean_recovered_per = []
bits = 256

for _ in range(1000):
    recovered_per = key_recovery_metrics(bits)
    mean_recovered_per.append(recovered_per)
    
mean(mean_recovered_per)

0.41571024930173567